----TODO-----
Actualizar feronomonas, sobre todo fitness por pieza
Pruebas Fitness

In [1]:
import copy
from typing import List, Dict

In [2]:
class PiezaClase:
    def __init__(self, pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos
    
"""     def _initialize(pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos """

'     def _initialize(pieza, x_pos, y_pos):\n        self.pieza = pieza\n        self.x_pos = x_pos\n        self.y_pos = y_pos '

In [3]:
def explore_adyacent(temp, x, y): #hay que marcar la celda inicial antes de llamarla
    area = 0
    # Verificar si la celda adyacente está dentro de los límites del espacio
    if x + 1 < len(temp) and temp[x+1][y] == 0:
        temp[x+1][y] = 1
        area += explore_adyacent(temp, x+1, y)
    if x - 1 >= 0 and temp[x-1][y] == 0:
        temp[x-1][y] = 1
        area += explore_adyacent(temp, x-1, y)
    if y + 1 < len(temp[0]) and temp[x][y+1] == 0:
        temp[x][y+1] = 1
        area += explore_adyacent(temp, x, y+1)
    if y - 1 >= 0 and temp[x][y-1] == 0:
        temp[x][y-1] = 1
        area += explore_adyacent(temp, x, y-1)
    if temp[x][y]:
        area += 1
    return area


def huecos (mat):
  matrix = copy.deepcopy(mat)
  array = []
  for i in range(len(mat)):
    for j in range(len(mat[0])):
      if matrix[i][j] == 0:
        matrix[i][j] = 1
        array.append(explore_adyacent(matrix, i, j))
  return array

space =  [[0, 1, 0, 1],
          [0, 1, 0, 1],
          [0, 0, 1, 0],
          [0, 1, 1, 1]]
temp_area = copy.deepcopy(space)
array = huecos(temp_area)
print(space)
print (temp_area)
print (array)

[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[5, 2, 1]


In [4]:
def clamp_to_zero(number):
    if number < 0:
        return 0
    else:
        return number

In [5]:
#funcion para imprimir una matriz mas esteticamente
def imprimir_matriz(matriz):
    # Encuentra la longitud máxima para cada columna
    max_lengths = [max(len(str(row[i])) for row in matriz) for i in range(len(matriz[0]))]

    # Imprime la matriz con los números alineados correctamente
    for row in matriz:
        for i, item in enumerate(row):
            print(str(item).rjust(max_lengths[i]), end=" ")
        print()  # Nueva línea después de cada fila


FITNESS DEFINITION

In [6]:
def fitnessSinPieza(space):
    espacios = huecos(space)
    value = 100
    value = value - len(espacios) * 5 #penalizar numero de huecos
    for hueco in espacios:
      value = value - (1/hueco) * 3
    return clamp_to_zero(value)

In [7]:
import copy

def colocar_pieza(space, pieza, x, y):
    temp = copy.deepcopy(space)  # Deep copy the space to avoid modifying the original
    piece_height = len(pieza)
    piece_width = len(pieza[0])
    #print("\n".join([" ".join(map(str, row)) for row in temp]))

    # Verifica si la pieza puede colocarse en las coordenadas (x, y)
    if x + piece_height > len(space) or y + piece_width > len(space[0]):
        #print("La pieza queda fuera del espacio, fitness deberia ser 0")
        return False  # Las coordenadas están fuera del espacio
    
    for i in range(piece_height):
        for j in range(piece_width):
            if space[x+i][y+j] == 1 :
                #print("La pieza se superpondría con otra en esta posición")
                return False  # La pieza se superpondría con otra pieza ya colocada
            temp[x+i][y+j] = pieza[i][j]  # Coloca la pieza en el espacio temporal
    
    # Copia el espacio temporal modificado de vuelta al espacio original
    for i in range(len(space)):
        for j in range(len(space[0])):
            space[i][j] = temp[i][j]
    
    return True  # La pieza se colocó con éxito en las coordenadas (x, y)

In [8]:
def fitness(space, pieza, x, y):
    temp = copy.deepcopy(space)  # Deep copy the space to avoid modifying the original
    valid = colocar_pieza(temp, pieza.pieza, x, y)
    espacios = huecos(temp)
    if not valid:
        #print("La pieza no se puede colocar en la posición dada, por lo que el fitness es 0.")
        return 0
    else:
        value = 100
        value -= len(espacios) * 5  # Penalizar el número de huecos
        #print(f"Número de huecos: {len(espacios)}")
        for hueco in espacios:
            value -= (1 / hueco) * 3
            #print(f"Hueco: {hueco}, Penalización: {(1/hueco) * 3}")
            #print(f"fitness value: {value}")
        return clamp_to_zero(value)


In [9]:
def obtener_fitness_para_todas_las_posiciones(space, pieza):
    fitness_matriz = [[0 for _ in range(len(space[0]))] for _ in range(len(space))]

    print("Hemos inicializado bien la matriz del fitness")

    for i in range(len(space)):
        for j in range(len(space[0])):
            temp_space = [row[:] for row in space]  # Copia profunda de la matriz original
            colocada = colocar_pieza(temp_space, pieza.pieza, i, j)  # Modificación aquí
            if colocada:
                #colocada va a ser true si se puede colocar la pieza en i,j
                #print("Empezamos a rellenar la matriz del fitness")
                #print("La pieza podría colocarse en esta posición, por lo que el fitness debe ser > 0.")
                fitness_matriz[i][j] = round (fitness(space, pieza, i, j), 2)  # Almacena el fitness sin la pieza
                #funcion de fitness ejecutada correctamente

    print("Hemos rellenado correctamente la matriz del fitness")
    # Imprime la matriz de fitness
    print("Matriz de fitness:")
    imprimir_matriz(fitness_matriz)
    

    return fitness_matriz

In [10]:
#Función para crear la pieza, empezamos creando piezas de un máximo de area de 3x3, para empezar con una versión sencilla del problema, haremos que las piezas inicialmente solo puedan ser cuadrados de 1x1, 2x2 o 3x3.

import random

def generar_pieza():
    # Inicializamos una matriz vacía
    sizex = random.randint(1, 6)
    sizey = random.randint(1, 6)
    pieza = [[1 for column in range(sizex)] for row in range(sizey)]
    return pieza


def imprimir_pieza(pieza:PiezaClase):
    for row in pieza.pieza:
        print("  ".join(map(str, row)))
    print("coordenadas: x:",pieza.x_pos, "y:",pieza.y_pos)
    print()

def imprimir_espacio(space):
    for row in space:
        print("  ".join(map(str, row)))
    print()



# Ejemplo de uso
if __name__ == "__main__":
    pieza = PiezaClase(generar_pieza(),random.randint(1, 6),random.randint(1, 6))
    imprimir_pieza(pieza)

1  1  1
1  1  1
1  1  1
1  1  1
1  1  1
1  1  1
coordenadas: x: 6 y: 3



TEST DE FITNESS:

In [ ]:
def test_fitness_function():
    # Genera un espacio pequeño para las pruebas
    print("Empezamos a generar el espacio.")
    x = 10
    y = 10
    space = [[0 for _ in range(x)] for _ in range(y)]

    # Genera un conjunto de piezas para las pruebas
    print("Procedemos a generar las piezas.")
    piezas = []
    for i in range(6):  # Limita el número de piezas para mantener las pruebas manejables
        # Genera una pieza individual con diferentes posiciones x e y en cada iteración
        piezaIndividual = PiezaClase(generar_pieza(), i, i)
        piezas.append(piezaIndividual)
    print("Piezas generadas correctamente.")    

    # Realiza las pruebas de la función de fitness para cada pieza
    print("Empezamos las pruebas.")
    for pieza in piezas:
        x_pos = pieza.x_pos
        y_pos = pieza.y_pos

        # Imprime la matriz de espacio antes de colocar la pieza
        print("Matriz de espacio antes de colocar la pieza:")
        imprimir_espacio(space)

        # Imprime la pieza antes de intentar colocarla
        print("Pieza a colocar:")
        imprimir_espacio(pieza.pieza)

        # Llama a la función para obtener el fitness para todas las posiciones de la matriz
        print("Llamamos a la funcion para obtener la matriz de fitness.")
        obtener_fitness_para_todas_las_posiciones(space, pieza)

        print("La matriz de fitness se ha rellenado correctamente \n")

        # Coloca la pieza en el espacio
        print("Comprobamos si podemos colocar la pieza:")
        colocada = False
        for i in range(len(space)):  # Recorre todas las posiciones del espacio
            for j in range(len(space[0])):
                colocada = colocar_pieza(space, pieza.pieza, i, j)
                if colocada:  # Si la pieza fue colocada, sale del bucle
                    print("Pieza colocada exitosamente.")
                    break
            if colocada:
                break  # Sale del bucle externo si la pieza fue colocada
        if not colocada:
            print("La pieza no se puede colocar en ninguna posición dada.")

        # Imprime la matriz de espacio después de colocar la pieza
        print("Matriz de espacio después de colocar la pieza:")
        imprimir_espacio(space)


test_fitness_function()


Funcion para decir las primeras coordenadas de una matriz (las más arriba a la izquierda) en las que quepa la pieza que se le ha pasado:

In [ ]:
def coordenadasEsquinaPosibles (space, pieza):

    temp = copy.deepcopy(space)  # Deep copy the space to avoid modifying the original
    for i in range(len(space)):  # Recorre todas las posiciones del espacio
            for j in range(len(space[0])):
                posible = colocar_pieza(temp, pieza.pieza)
                if posible:
                    return [i, j]
    
